In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

import os, shutil
from glob import glob

from tensorflow import keras
from keras import layers
from tensorflow_addons.optimizers import AdamW

from PIL import Image

Основной целью является итеративно с вещественным шагом (0-1) зашумлять изображение и производя обратный процесс нучить модель предсказывать шум (отделять шумный компонент) в изображении. Модель получает случайный шум $\sim \mathcal{N}(0, 1)$ на вход и итеративно устраняет шум, оставляя сгенерированное изображение.

In [ ]:
image = Image.open("../input/celeba-dataset/img_align_celeba/img_align_celeba/000001.jpg")
plt.imshow(image)
np.array(image).shape

In [ ]:
# if not os.path.exists("celeba_sampled"):
#     os.makedirs("celeba_sampled")

# all_images = glob("../input/celeba-dataset/img_align_celeba/img_align_celeba/*.jpg")

In [ ]:
# sampled_idx = np.random.choice(len(all_images), size=25000)
# all_images = np.array(all_images)
# all_images = all_images[sampled_idx]

In [ ]:
# cur = os.getcwd() + "/"

# i = 0
# for path in all_images:
#     path = str(path)
#     try:
#         shutil.move(str(path), "celeba_sampled/"+str(path).split("/")[-1])
#         i+=1
#     except FileNotFoundError:
#         continue
#     except OSError:
#         continue
# print(f"{len(os.listdir('celeba_sampled/'))} files replaced")

**Устанавливаем гиперпараметры**

v1+: lr 1e-4, select 40000 ims, repeat 2,

v2+: lr 1e-4, select 60000 ims, epochs 80, repeat 2

v3+: lr 1e-3, select 40000, repeat 3 + scheduler-patience 8 epochs 80

* best quality v4+: mse -> mae 40000, repeat 3, 1e-3 epochs 65, val 90%

v5: =v4 (mae, epochs=65, repeat=3, nsamp=40000, lr=1e-3+scheduler) + block_depth = 3

v6: best + upsample -> conv2d_transpose in upsample_block, epochs = 60

In [ ]:
# данные
image_directory = "../input/celeba-dataset/img_align_celeba/img_align_celeba/" # весь набор
# image_directory = "celeba_sampled/"

dataset_repetitions = 3  # выбрать 1-5, 5 для маленького набора
num_epochs = 60  # 50
image_size = 64

# KID = Kernel Inception Distance, метрика для измерения качества сгенерированных изображений по отношению к оригинальным
kid_image_size = 75  # размер изображения для подачи в Inception (она генерирует фичи для вычисления метрики)
kid_diffusion_steps = 5  #  количество итераций генерации: чем выше, тем точнее измерение
plot_diffusion_steps = 30  # (20) то же самое, но используется во время отрисовки результатов в процессе обучения,  >> повышает качетсво и осмысленность

# sampling
min_signal_rate = 0.02  # минимальное и максимальное значения noise rate and signal rate: стандартное отклонение в зашумленных изображениях, 
max_signal_rate = 0.95  # 

# architecture
embedding_dims = 32  # создается позиционный embedding из дисперсии шума (компонента зашумленного изображения), подаваемого на вход сети 2м входом
embedding_max_frequency = 1000.0
widths = [32, 64, 96, 128]  # количество фильтров в сверточных слоях
block_depth = 2  # 2, количество блоков в UNET сети, для данного набора лучше взять > 2, но железо не потянет

# optimization
batch_size = 64
ema = 0.999  # коэффициент для использования (вычисления экспоненциальных средних весов) клона основной сети на инференсе с трансформированными весами
learning_rate = 1e-3  # 1e-3 by default
weight_decay = 1e-4

In [ ]:
all_images = glob("../input/celeba-dataset/img_align_celeba/img_align_celeba/*.jpg")
sampled_idx = np.random.choice(len(all_images), size=40000)
all_images = np.array(all_images)[sampled_idx]
border = int(len(all_images) * 0.9)
train_images = all_images[:border]
valid_images = all_images[border:]

In [ ]:
def train_generator():
    for path in train_images:
        image = np.array(Image.open(path))
        yield tf.cast(image, dtype=tf.float32)
        
def valid_generator():
    for path in valid_images:
        image = np.array(Image.open(path))
        yield tf.cast(image, dtype=tf.float32)

**Аугментация**

Если будем применять аугментацию (что еще увеличит время обучения) - то лучше выбрать horizontal flips (повысит качетсво, не исказив результаты)

In [ ]:
def get_augmenter(image_size, uncropped_image_size=(64,64,3)):
    return keras.Sequential(
        [
            keras.Input(shape=uncropped_image_size),
            layers.Normalization(),
            layers.RandomFlip(mode="horizontal"),
            layers.RandomCrop(height=image_size, width=image_size),
        ],
        name="augmenter",
    )

def preprocess_image(data, crop_size=140):
    # вырезаем центр изображения
    height = 218
    width = 178
    image = tf.image.crop_to_bounding_box(
        data,
        (height - crop_size) // 2,
        (width - crop_size) // 2,
        crop_size,
        crop_size,
    )
    # изменяем размер изображения и нормализуем
    image = tf.image.resize(image, size=[image_size, image_size], method="bicubic", antialias=True)
    return tf.clip_by_value(image / 255.0, 0.0, 1.0)

def prepare_dataset(split, split_size=0.2):
    # shuffle обязательно - требуется для вычисления KID
    return (
        tf.data.Dataset.from_generator(train_generator, output_shapes=(218,178,3), output_types=tf.float32)
        .map(preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
        .cache()
        .repeat(dataset_repetitions)
        .shuffle(10 * batch_size)
        .batch(batch_size, drop_remainder=True)
        .prefetch(buffer_size=tf.data.AUTOTUNE)
    )

train_dataset = prepare_dataset("training")
val_dataset = prepare_dataset("validation")

In [ ]:
for p in train_dataset.take(1):
    print(p.shape)

image = p[np.random.randint(batch_size)].numpy()
plt.imshow(image)

In [ ]:
class KID(keras.metrics.Metric):
    def __init__(self, name, **kwargs):
        super().__init__(name=name, **kwargs)
        # KID - своего рода полиномиальные метрика по вычислению расстояния между двумя распределениями (изображениями)
        # KID вычисляется на каждом пакете и усредняется по эпохе
        self.kid_tracker = keras.metrics.Mean(name="kid_tracker")

        # для получения сверток изображений используем InceptionV3 без головы
        # пиксели варьируются (0-255), далее те же преобрахзования как и в датасете
        self.encoder = keras.Sequential(
            [
                keras.Input(shape=(image_size, image_size, 3)),
                layers.Rescaling(255.0),
                layers.Resizing(height=kid_image_size, width=kid_image_size),
                layers.Lambda(keras.applications.inception_v3.preprocess_input),
                keras.applications.InceptionV3(
                    include_top=False,
                    input_shape=(kid_image_size, kid_image_size, 3),
                    weights="imagenet",
                ),
                layers.GlobalAveragePooling2D(),
            ],
            name="inception_encoder",
        )

    def polynomial_kernel(self, features_1, features_2):
        feature_dimensions = tf.cast(tf.shape(features_1)[1], dtype=tf.float32)
        return (features_1 @ tf.transpose(features_2) / feature_dimensions + 1.0) ** 3.0

    def update_state(self, real_images, generated_images, sample_weight=None):
        real_features = self.encoder(real_images, training=False)
        generated_features = self.encoder(generated_images, training=False)

        # вычисляем полиномиальные ядра для двух фичей (таргета и сгенерировнных)
        kernel_real = self.polynomial_kernel(real_features, real_features)
        kernel_generated = self.polynomial_kernel(
            generated_features, generated_features
        )
        kernel_cross = self.polynomial_kernel(real_features, generated_features)

        # вычисляем квадратичное максимальное "несоответствие" используя средние значения ядер
        batch_size = tf.shape(real_features)[0]
        batch_size_f = tf.cast(batch_size, dtype=tf.float32)
        mean_kernel_real = tf.reduce_sum(kernel_real * (1.0 - tf.eye(batch_size))) / (
            batch_size_f * (batch_size_f - 1.0)
        )
        mean_kernel_generated = tf.reduce_sum(
            kernel_generated * (1.0 - tf.eye(batch_size))
        ) / (batch_size_f * (batch_size_f - 1.0))
        mean_kernel_cross = tf.reduce_mean(kernel_cross)
        kid = mean_kernel_real + mean_kernel_generated - 2.0 * mean_kernel_cross

        self.kid_tracker.update_state(kid)

    def result(self):
        return self.kid_tracker.result()

    def reset_state(self):
        self.kid_tracker.reset_state()

In [ ]:
# позиционный эмбеддинг для применения к шумовым дисперсиям, подаваемым на вход сети вместе с зашумленными изображениями
# используем только на входе для простоты (хотя в научных работах данное преобразование применяется несколько раз при проходе по сети)
def sinusoidal_embedding(x):
    embedding_min_frequency = 1.0
    frequencies = tf.exp(
        tf.linspace(
            tf.math.log(embedding_min_frequency),
            tf.math.log(embedding_max_frequency),
            embedding_dims // 2,
        )
    )
    angular_speeds = 2.0 * np.pi * frequencies
    embeddings = tf.concat(
        [tf.sin(angular_speeds * x), tf.cos(angular_speeds * x)], axis=3
    )
    return embeddings

# блок обратной связи для UNET
def ResidualBlock(width):
    def apply(x):
        input_width = x.shape[3]
        if input_width == width:
            residual = x
        else:
            residual = layers.Conv2D(width, kernel_size=1)(x)
        x = layers.BatchNormalization(center=False, scale=False)(x)  # ввиду дальнейших сверток обучаемые параметры определять не требуется
        x = layers.Conv2D(
            width, kernel_size=3, padding="same", activation=keras.activations.swish
        )(x)
        x = layers.Conv2D(width, kernel_size=3, padding="same")(x)
        x = layers.Add()([x, residual])
        return x

    return apply

# блок левой ветки
def DownBlock(width, block_depth):
    def apply(x):
        x, skips = x
        for _ in range(block_depth):
            x = ResidualBlock(width)(x)
            skips.append(x)
        x = layers.AveragePooling2D(pool_size=2)(x)
        return x

    return apply

# блок правой ветки
def UpBlock(width, block_depth):
    def apply(x):
        x, skips = x
        x = layers.UpSampling2D(size=2, interpolation="bilinear")(x)
        # x = layers.Conv2DTranspose(width, 3, strides=2, padding="same", activation=keras.activations.swish)(x)  # activation ?
        for _ in range(block_depth):
            x = layers.Concatenate()([x, skips.pop()])
            x = ResidualBlock(width)(x)
        return x

    return apply

# архитектура UNET в сборе
def get_network(image_size, widths, block_depth):
    noisy_images = keras.Input(shape=(image_size, image_size, 3))
    noise_variances = keras.Input(shape=(1, 1, 1))

    e = layers.Lambda(sinusoidal_embedding)(noise_variances)
    e = layers.UpSampling2D(size=image_size, interpolation="nearest")(e)

    x = layers.Conv2D(widths[0], kernel_size=1)(noisy_images)
    x = layers.Concatenate()([x, e])

    skips = []
    for width in widths[:-1]:
        x = DownBlock(width, block_depth)([x, skips])

    for _ in range(block_depth):
        x = ResidualBlock(widths[-1])(x)

    for width in reversed(widths[:-1]):
        x = UpBlock(width, block_depth)([x, skips])

    x = layers.Conv2D(3, kernel_size=1, kernel_initializer="zeros")(x)  # предсказание на первых этапах будет средним по таргету, лосс = 1

    return keras.Model([noisy_images, noise_variances], x, name="residual_unet")

In [ ]:
class DiffusionModel(keras.Model):
    def __init__(self, image_size, widths, block_depth, augmenter):
        super().__init__()

        self.augmenter = augmenter
        self.normalizer = layers.Normalization()  # слой нормализации
        self.network = get_network(image_size, widths, block_depth)  # сеть UNET - основа
        self.ema_network = keras.models.clone_model(self.network)  # сеть для инфереснса с трансформированными весами основной сети

    def compile(self, **kwargs):  # метрика
        super().compile(**kwargs)

        self.noise_loss_tracker = keras.metrics.Mean(name="n_loss")
        self.image_loss_tracker = keras.metrics.Mean(name="i_loss")
        self.kid = KID(name="kid")

    @property
    def metrics(self):
        return [self.noise_loss_tracker, self.image_loss_tracker, self.kid]

    def denormalize(self, images):
        # преобразуем пиксели к диапазонк (0-1)
        images = self.augmenter.layers[0].mean + (
            images * self.augmenter.layers[0].variance ** 0.5
        )
        # images = self.normalizer.mean + images * self.normalizer.variance**0.5  # без аугментации
        return tf.clip_by_value(images, 0.0, 1.0)

    def diffusion_schedule(self, diffusion_times):
        # возвращает на заданном шаге уровень шума и сигнала
        start_angle = tf.acos(max_signal_rate)
        end_angle = tf.acos(min_signal_rate)

        diffusion_angles = start_angle + diffusion_times * (end_angle - start_angle)

        # углы - вычисляем оценку шума и сигнала (их сумма квадратов дает 1 по основному тригонометрическому правилу)
        signal_rates = tf.cos(diffusion_angles)
        noise_rates = tf.sin(diffusion_angles)

        return noise_rates, signal_rates

    def denoise(self, noisy_images, noise_rates, signal_rates, training):
        # экспоненциальные скользящие средние весов используются при инференсе
        if training:
            network = self.network
        else:
            network = self.ema_network

        # предсказываем шум и используем его для вычисления изображения
        pred_noises = network([noisy_images, noise_rates**2], training=training)
        pred_images = (noisy_images - noise_rates * pred_noises) / signal_rates

        return pred_noises, pred_images

    def reverse_diffusion(self, initial_noise, diffusion_steps):
        # обратный процесс - дешумизация
        num_images = initial_noise.shape[0]
        step_size = 1.0 / diffusion_steps

        # на начальном этапе зашумленное изображение - это чистый шум
        # однако уровень полезного сигнала данного изображение = min_signal_rate
        next_noisy_images = initial_noise
        for step in range(diffusion_steps):
            noisy_images = next_noisy_images

            # разделяем зашумленное изображение на полезный сигнал и шум
            diffusion_times = tf.ones((num_images, 1, 1, 1)) - step * step_size
            noise_rates, signal_rates = self.diffusion_schedule(diffusion_times)
            pred_noises, pred_images = self.denoise(
                noisy_images, noise_rates, signal_rates, training=False
            )

            # получаем предсказаныне компоненты (шум-сигнал) используя уровень шума и сигнала на следующем шаге
            # используем новое полученное зашумленное изображение на следующем шаге (изображение стало чуть чище)
            next_diffusion_times = diffusion_times - step_size
            next_noise_rates, next_signal_rates = self.diffusion_schedule(
                next_diffusion_times
            )
            next_noisy_images = (
                next_signal_rates * pred_images + next_noise_rates * pred_noises
            )

        return pred_images

    def generate(self, num_images, diffusion_steps):
        # шум -> изображение -> получение денормализованных значений пикселей
        initial_noise = tf.random.normal(shape=(num_images, image_size, image_size, 3))
        generated_images = self.reverse_diffusion(initial_noise, diffusion_steps)
        generated_images = self.denormalize(generated_images)
        return generated_images

    def train_step(self, images):  # главный шаг обучения
        # нормализуем изображения, чтобы получить стандартное отклонение = 1, прямо как у шума
        images = self.augmenter(images, training=True)
        # images = self.normalizer(images, training=True)  # без аугментации
        noises = tf.random.normal(shape=(batch_size, image_size, image_size, 3))

        # генерируем случайное количество шагов
        diffusion_times = tf.random.uniform(
            shape=(batch_size, 1, 1, 1), minval=0.0, maxval=1.0
        )
        noise_rates, signal_rates = self.diffusion_schedule(diffusion_times)  # уровень шума и сигнала
        # перемешиваем изображения с шумом
        noisy_images = signal_rates * images + noise_rates * noises

        with tf.GradientTape() as tape:
            # обучаем сеть разделять изображение на шум и полезный сигнал (незашумленное изображение)
            pred_noises, pred_images = self.denoise(
                noisy_images, noise_rates, signal_rates, training=True
            )

            noise_loss = self.loss(noises, pred_noises)  # лосс при предсказании шума используем для обновления весов
            image_loss = self.loss(images, pred_images)  # лосс на сгенерированном изображении используем для вычисления метрики

        gradients = tape.gradient(noise_loss, self.network.trainable_weights)
        self.optimizer.apply_gradients(zip(gradients, self.network.trainable_weights))

        self.noise_loss_tracker.update_state(noise_loss)
        self.image_loss_tracker.update_state(image_loss)

        # после вычисления весов при обратном распространении ошибки трансфомрируем веса второй модели через экспоненциальное скользящее среднее
        for weight, ema_weight in zip(self.network.weights, self.ema_network.weights):
            ema_weight.assign(ema * ema_weight + (1 - ema) * weight)

        # метрика не вычисляется на этапе оптимизации для ускорения процесса (прогнать два изображения через сверточную сеть и произвести вычисление долго)
        return {m.name: m.result() for m in self.metrics[:-1]}

    def test_step(self, images):
        # нормализуем изображения, чтобы получить стандартное отклонение = 1, прямо как у шума
        images = self.augmenter(images, training=False)
        # images = self.normalizer(images, training=False)  # без аугментации
        noises = tf.random.normal(shape=(batch_size, image_size, image_size, 3))

        # генерируем случайное количество шагов
        diffusion_times = tf.random.uniform(
            shape=(batch_size, 1, 1, 1), minval=0.0, maxval=1.0
        )
        noise_rates, signal_rates = self.diffusion_schedule(diffusion_times)
        # перемешиваем изображение и шум
        noisy_images = signal_rates * images + noise_rates * noises

        # разделяем зашумленные изображения на полезный сигнал и шум
        pred_noises, pred_images = self.denoise(
            noisy_images, noise_rates, signal_rates, training=False
        )

        noise_loss = self.loss(noises, pred_noises)
        image_loss = self.loss(images, pred_images)

        self.image_loss_tracker.update_state(image_loss)
        self.noise_loss_tracker.update_state(noise_loss)

        # несколько раз вычисляем метрику
        images = self.denormalize(images)
        generated_images = self.generate(
            num_images=batch_size, diffusion_steps=kid_diffusion_steps
        )
        self.kid.update_state(images, generated_images)

        return {m.name: m.result() for m in self.metrics}

    def plot_images(self, epoch=None, logs=None, num_rows=3, num_cols=6):
        # функция для отрисовки сгенерированных изображений
        generated_images = self.generate(
            num_images=num_rows * num_cols,
            diffusion_steps=plot_diffusion_steps,
        )

        plt.figure(figsize=(num_cols * 2.0, num_rows * 2.0))
        for row in range(num_rows):
            for col in range(num_cols):
                index = row * num_cols + col
                plt.subplot(num_rows, num_cols, index + 1)
                plt.imshow(generated_images[index])
                plt.axis("off")
        plt.tight_layout()
        plt.show()
        plt.close()

Также попробуем:
* использовать подрезку весов;
* mean average error вместо mean squared error
* использовать обратную свертку вместо апсэмплинга (добавим обучаемые параметры: апсэмплинг это интерполяция, обратная свертка это обучение с весами).

In [ ]:
# создаем модель
# model = DiffusionModel(image_size, widths, block_depth,)  # без аугментации
model = DiffusionModel(image_size, widths, block_depth, augmenter=get_augmenter(image_size=image_size),)


# используем AdamW, более продвинутую версию Adam. Позволяет также динамически изменять скорость обучения для каждого параметра, а также применять
# более продвинутую регуляризацию
model.compile(
    optimizer=AdamW(
        learning_rate=learning_rate, weight_decay=weight_decay
    ),
    loss = keras.losses.mean_absolute_error
    # loss=keras.losses.mean_squared_error,
)

checkpoint_path = "checkpoints/diffusion_model"
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    save_weights_only=True,
    monitor="val_kid",
    mode="min",
    save_best_only=True,
)

# вычислим среднее и дисперсию тренировочного набора и используем их в нормализовочном слое сети
# model.normalizer.adapt(train_dataset)  # без аугментации
model.augmenter.layers[0].adapt(train_dataset)

callbacks = [
        keras.callbacks.ReduceLROnPlateau(monitor="val_n_loss", mode="min", patience=8),  # for 3+ experiment
        keras.callbacks.LambdaCallback(on_epoch_end=model.plot_images),
        checkpoint_callback,
    ]

model.fit(train_dataset, epochs=num_epochs, validation_data=val_dataset, callbacks=callbacks, verbose=1)

Вещь очень полезная на практике: например, используется в DALLE-2 для генерации картики из текста (причем, сразу две: сначала генерируется небольшое изображение, а из него большое), только сэмлится не из шума, а из текстового эмбеддинга, который в свою очередь получен из авторегрессионной модеои и еще одно diffudion модели. В них же информация (текстовый эбеддинг и эмбеддинг картинки) поступают из модели CLIP. Также они задействованы в  google prompt-to-prompt модели: генерирует картинку из текста, но при этом берется исходный текст, в него вносятся правки и на выходе получается картика с учетом нового текста.